# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 6 2023 8:30AM,258656,10,PRF-54686,Bio-PRF,Released
1,Mar 6 2023 8:30AM,258656,10,PRF-54691,Bio-PRF,Released
2,Mar 6 2023 8:30AM,258656,10,PRF-54694,Bio-PRF,Released
3,Mar 6 2023 8:30AM,258656,10,PRF-54697,Bio-PRF,Released
4,Mar 6 2023 8:30AM,258657,10,Enova-11461,Emerginnova,Released
5,Mar 6 2023 8:30AM,258657,10,Enova-11462,Emerginnova,Released
6,Mar 3 2023 3:20PM,258658,16,SHL-9227526,"SHL Pharma, LLC",Released
7,Mar 3 2023 2:44PM,258655,10,9228151,Bio-PRF,Executing
8,Mar 3 2023 2:44PM,258655,10,9228152,Bio-PRF,Released
9,Mar 3 2023 2:44PM,258655,10,9228154,Bio-PRF,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,258655,Executing,3
20,258655,Released,4
21,258656,Released,4
22,258657,Released,2
23,258658,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258654,NaN,1.0,NaN
258655,NaN,3.0,4.0
258656,NaN,NaN,4.0
258657,NaN,NaN,2.0
258658,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258617,0.0,0.0,1.0
258618,0.0,0.0,1.0
258622,2.0,8.0,1.0
258623,0.0,0.0,1.0
258624,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258617,0,0,1
258618,0,0,1
258622,2,8,1
258623,0,0,1
258624,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258617,0,0,1
1,258618,0,0,1
2,258622,2,8,1
3,258623,0,0,1
4,258624,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258617,,,1
1,258618,,,1
2,258622,2,8,1
3,258623,,,1
4,258624,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 6 2023 8:30AM,258656,10,Bio-PRF
4,Mar 6 2023 8:30AM,258657,10,Emerginnova
6,Mar 3 2023 3:20PM,258658,16,"SHL Pharma, LLC"
7,Mar 3 2023 2:44PM,258655,10,Bio-PRF
14,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC
15,Mar 3 2023 2:20PM,258652,10,Emerginnova
16,Mar 3 2023 2:15PM,258651,10,ISDIN Corporation
17,Mar 3 2023 2:06PM,258650,15,"Alliance Pharma, Inc."
18,Mar 3 2023 2:05PM,258649,15,"Person & Covey, Inc."
26,Mar 3 2023 2:01PM,258648,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 6 2023 8:30AM,258656,10,Bio-PRF,,,4
1,Mar 6 2023 8:30AM,258657,10,Emerginnova,,,2
2,Mar 3 2023 3:20PM,258658,16,"SHL Pharma, LLC",,,1
3,Mar 3 2023 2:44PM,258655,10,Bio-PRF,,3,4
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,,1,
5,Mar 3 2023 2:20PM,258652,10,Emerginnova,,,1
6,Mar 3 2023 2:15PM,258651,10,ISDIN Corporation,,,1
7,Mar 3 2023 2:06PM,258650,15,"Alliance Pharma, Inc.",,,1
8,Mar 3 2023 2:05PM,258649,15,"Person & Covey, Inc.",,,8
9,Mar 3 2023 2:01PM,258648,15,"Alliance Pharma, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4,,
1,Mar 6 2023 8:30AM,258657,10,Emerginnova,2,,
2,Mar 3 2023 3:20PM,258658,16,"SHL Pharma, LLC",1,,
3,Mar 3 2023 2:44PM,258655,10,Bio-PRF,4,3,
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,,1,
5,Mar 3 2023 2:20PM,258652,10,Emerginnova,1,,
6,Mar 3 2023 2:15PM,258651,10,ISDIN Corporation,1,,
7,Mar 3 2023 2:06PM,258650,15,"Alliance Pharma, Inc.",1,,
8,Mar 3 2023 2:05PM,258649,15,"Person & Covey, Inc.",8,,
9,Mar 3 2023 2:01PM,258648,15,"Alliance Pharma, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4,,
1,Mar 6 2023 8:30AM,258657,10,Emerginnova,2,,
2,Mar 3 2023 3:20PM,258658,16,"SHL Pharma, LLC",1,,
3,Mar 3 2023 2:44PM,258655,10,Bio-PRF,4,3,
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4.0,NaN,NaN
1,Mar 6 2023 8:30AM,258657,10,Emerginnova,2.0,NaN,NaN
2,Mar 3 2023 3:20PM,258658,16,"SHL Pharma, LLC",1.0,NaN,NaN
3,Mar 3 2023 2:44PM,258655,10,Bio-PRF,4.0,3.0,NaN
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 6 2023 8:30AM,258656,10,Bio-PRF,4.0,0.0,0.0
1,Mar 6 2023 8:30AM,258657,10,Emerginnova,2.0,0.0,0.0
2,Mar 3 2023 3:20PM,258658,16,"SHL Pharma, LLC",1.0,0.0,0.0
3,Mar 3 2023 2:44PM,258655,10,Bio-PRF,4.0,3.0,0.0
4,Mar 3 2023 2:32PM,258654,12,HealthCaps LLC,0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1810532,15.0,22.0,2.0
12,517282,0.0,2.0,0.0
15,775947,10.0,0.0,0.0
16,517305,2.0,0.0,0.0
19,1293128,5.0,5.0,0.0
20,258627,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1810532,15.0,22.0,2.0
1,12,517282,0.0,2.0,0.0
2,15,775947,10.0,0.0,0.0
3,16,517305,2.0,0.0,0.0
4,19,1293128,5.0,5.0,0.0
5,20,258627,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,15.0,22.0,2.0
1,12,0.0,2.0,0.0
2,15,10.0,0.0,0.0
3,16,2.0,0.0,0.0
4,19,5.0,5.0,0.0
5,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,15.0
1,12,Released,0.0
2,15,Released,10.0
3,16,Released,2.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,22.0,2.0,0.0,0.0,5.0,0.0
Released,15.0,0.0,10.0,2.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,22.0,2.0,0.0,0.0,5.0,0.0
2,Released,15.0,0.0,10.0,2.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,22.0,2.0,0.0,0.0,5.0,0.0
2,Released,15.0,0.0,10.0,2.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()